# BAM Processing

> Processing BAM files for barcode extraction and classification

## Introduction
This notebook implements the BAM processing functionality for BarcodeSeqKit. It includes functions for handling BAM files, extracting barcodes, and sorting reads into appropriate output files based on barcode detection.

In [ ]:
#| default_exp bam_processing

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import subprocess
import logging
from typing import List, Dict, Tuple, Optional, Iterator, Any
import pysam
from tqdm.auto import tqdm

from BarcodeSeqKit.core import (
    BarcodeConfig, 
    BarcodeMatch, 
    OrientationType,
    ExtractionStatistics,
    BarcodeLocationType,
    BarcodeSeqKitConfig,
    print_extraction_summary,
    save_stats_and_config
)
from BarcodeSeqKit.sequence_utils import (
    classify_read_by_first_match,
    extract_softclipped_region,
    prepare_categories
)

## BAM File Utilities
First, let's define utility functions for BAM file operations.

In [ ]:
#| export
class BamUtils:
    """Utility functions for BAM file operations."""
    
    @staticmethod
    def sort_and_index(bam_file: str) -> None:
        """Sort and index a BAM file by reference.
        
        Args:
            bam_file: Path to the BAM file
        """
        # Create a temporary file for sorting
        temp_file = f"{bam_file}.temp.bam"
        
        # Sort the BAM file to a temporary file
        pysam.sort("-o", temp_file, bam_file)
        
        # Replace the original with the sorted file
        os.replace(temp_file, bam_file)
        
        # Index the BAM file
        pysam.index(bam_file)
    
    @staticmethod
    def get_read_count(bam_file: str) -> int:
        """Get the total number of reads in a BAM file.
        
        Args:
            bam_file: Path to the BAM file
            
        Returns:
            Total number of reads
        """
        # Run samtools idxstats to get read counts
        cmd = ["samtools", "idxstats", bam_file]
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode != 0:
            raise RuntimeError(f"Error running samtools idxstats: {result.stderr}")
        
        # Parse the output
        mapped = 0
        unmapped = 0
        for line in result.stdout.strip().split("\n"):
            if not line:
                continue
            parts = line.split("\t")
            if len(parts) >= 4:
                mapped += int(parts[2])
                unmapped += int(parts[3])
        
        return mapped + unmapped
    
    @staticmethod
    def merge_bam_files(output_file: str, input_files: List[str]) -> None:
        """Merge multiple BAM files into one.
        
        Args:
            output_file: Path to the output BAM file
            input_files: List of input BAM file paths
        """
        if not input_files:
            return
        
        # Use pysam to merge the files
        pysam.merge("-f", output_file, *input_files)

## BAM Processing Functions
Now let's implement the main functions for processing BAM files.

In [ ]:
#| export
def validate_bam_file(bam_file: str, config: BarcodeSeqKitConfig) -> str:
    """Validate and prepare a BAM file for processing.
    
    Args:
        bam_file: Path to the BAM file
        config: BarcodeSeqKit configuration
        
    Returns:
        Path to the validated BAM file
        
    Raises:
        FileNotFoundError: If the BAM file doesn't exist
    """
    # Check if the file exists
    if not os.path.exists(bam_file):
        raise FileNotFoundError(f"BAM file not found: {bam_file}")
    
    # Check if the BAM file is sorted and indexed
    try:
        # Try to open the index file
        pysam.AlignmentFile(bam_file, "rb").check_index()
        is_indexed = True
    except (ValueError, IOError):
        config.logger.warning(f"BAM file is not indexed: {bam_file}")
        config.logger.info("Sorting and indexing BAM file...")
        BamUtils.sort_and_index(bam_file)
        is_indexed = True
    
    return bam_file

In [ ]:
#| export
def get_output_path(category: str, config: BarcodeSeqKitConfig) -> str:
    """Get the output path for a category.
    
    Args:
        category: Category name
        config: BarcodeSeqKit configuration
        
    Returns:
        Path to the output BAM file
    """
    filename = f"{config.output_prefix}_{category}.bam"
    return os.path.join(config.output_dir, filename)

In [ ]:
#| export
def init_output_files(template_file: pysam.AlignmentFile, categories: List[str], config: BarcodeSeqKitConfig) -> Dict[str, pysam.AlignmentFile]:
    """Initialize output BAM files.
    
    Args:
        template_file: Template BAM file
        categories: List of output categories
        config: BarcodeSeqKit configuration
        
    Returns:
        Dictionary mapping categories to output file handles
    """
    output_files = {}
    
    for category in categories:
        output_path = get_output_path(category, config)
        config.logger.debug(f"Creating output file for category {category}: {output_path}")
        output_files[category] = pysam.AlignmentFile(
            output_path, "wb", template=template_file
        )
    
    return output_files

In [ ]:
#| export
def classify_read(sequence: str, config: BarcodeSeqKitConfig, single_barcode_mode: bool) -> Tuple[Optional[BarcodeMatch], str]:
    """Classify a read sequence based on barcode matches.
    
    Args:
        sequence: Read sequence to classify
        config: BarcodeSeqKit configuration
        single_barcode_mode: Whether we're in single barcode mode
        
    Returns:
        Tuple of (best_match, category)
    """
    if not sequence:
        return None, "noBarcode"
    
    sequence = sequence.upper()
    match, original_category = classify_read_by_first_match(
        sequence=sequence,
        barcodes=config.barcodes,
        max_mismatches=config.max_mismatches
    )
    
    # Adjust category based on single barcode mode or specific location mode
    if match:
        if single_barcode_mode:
            # For single barcode mode, use simpler categories
            if match.orientation == OrientationType.FORWARD:
                return match, "barcode_orientFR"
            else:  # REVERSE_COMPLEMENT
                return match, "barcode_orientRC"
        else:
            # For multiple barcodes with locations, use the location in the category
            location = match.barcode.location.value
            if location in ["5", "3"]:
                orientation = match.orientation.value
                return match, f"barcode{location}_orient{orientation}"
    
    return match, "noBarcode"

In [ ]:
#| export
def process_bam_file(bam_file: str, config: BarcodeSeqKitConfig) -> ExtractionStatistics:
    """Process a BAM file for barcode extraction.
    
    Args:
        bam_file: Path to the BAM file
        config: BarcodeSeqKit configuration
        
    Returns:
        ExtractionStatistics object with results
    """
    # Validate and prepare the BAM file
    bam_file = validate_bam_file(bam_file, config)
    
    # Determine if we're in single barcode mode
    single_barcode_mode = len(config.barcodes) == 1 or all(b.location.value == "UNK" for b in config.barcodes)
    
    # Prepare output categories
    categories = prepare_categories(config.barcodes, config.keep_unmatched)
    
    # Get total read count for progress reporting
    total_reads = BamUtils.get_read_count(bam_file)
    config.logger.info(f"BAM file: {bam_file} ({total_reads} reads)")
    config.logger.info(f"Output categories: {categories}")
    
    # Initialize statistics
    stats = ExtractionStatistics()
    stats.total_reads = total_reads
    
    # Open the input BAM file
    bamfile = pysam.AlignmentFile(bam_file, "rb")
    
    # Initialize output BAM files
    output_files = init_output_files(bamfile, categories, config)
    
    # Track processed reads to avoid duplicates
    read_classifications = {}
    
    try:
        # First pass: classify reads by barcode
        config.logger.info("First pass: classifying reads by barcode")
        for read in tqdm(bamfile, total=total_reads, desc="Classifying reads"):
            # Skip if we've already processed this read
            if read.query_name in read_classifications:
                continue
            
            # Get the read sequence
            if config.search_softclipped:
                sequence = extract_softclipped_region(read)
            else:   
                sequence = read.query_sequence
            
            # Search for barcodes
            match, category = classify_read(sequence, config, single_barcode_mode)
            
            # Update statistics and classification
            if match:
                stats.update_barcode_match(match, category)
                read_classifications[read.query_name] = category
            else:
                stats.no_barcode_count += 1
                if config.keep_unmatched:
                    read_classifications[read.query_name] = "noBarcode"
        
        config.logger.info(f"First pass complete: classified {len(read_classifications)} reads")
        
        # Reset for second pass
        bamfile.close()
        bamfile = pysam.AlignmentFile(bam_file, "rb")
        
        # Second pass: write reads to output files
        config.logger.info("Second pass: writing reads to output files")
        for read in tqdm(bamfile, total=total_reads, desc="Writing output files"):
            category = read_classifications.get(read.query_name)
            
            # Write to output file if the read has a category
            if category and category in output_files:
                output_files[category].write(read)
        
    finally:
        # Close all file handles
        bamfile.close()
        for category, f in output_files.items():
            config.logger.debug(f"Closing output file for category {category}")
            f.close()
    
    # Sort and index all output files
    config.logger.info("Sorting and indexing output files")
    for category in output_files:
        file_path = get_output_path(category, config)
        config.logger.debug(f"Sorting and indexing {file_path}")
        BamUtils.sort_and_index(file_path)
    
    # Print summary
    print_extraction_summary(stats, config)
    
    # Save statistics and configuration
    save_stats_and_config(stats, config)
    
    return stats

## Main BAM Processing Function
Now let's define a main function that ties everything together.

In [ ]:
#| export
def extract_barcodes_from_bam(bam_file: str, config: BarcodeSeqKitConfig) -> ExtractionStatistics:
    """Extract barcodes from a BAM file.
    
    This is the main entry point for BAM processing.
    
    Args:
        bam_file: Path to the BAM file
        config: BarcodeSeqKit configuration
        
    Returns:
        ExtractionStatistics with processing results
    """
    config.logger.info(f"Starting barcode extraction from BAM file: {bam_file}")
    config.logger.info(f"Output prefix: {config.output_prefix}")
    config.logger.info(f"Output directory: {config.output_dir}")
    config.logger.info(f"Number of barcodes: {len(config.barcodes)}")
    
    try:
        # Process the BAM file
        stats = process_bam_file(bam_file, config)
        config.logger.info("BAM processing completed successfully")
        return stats
    except Exception as e:
        config.logger.error(f"Error processing BAM file: {str(e)}")
        import traceback
        config.logger.error(traceback.format_exc())
        raise

## Example Usage
Let's demonstrate how to use these functions.

In [ ]:
# Example usage of BAM processing
# Note: This requires a real BAM file to run

from BarcodeSeqKit.core import BarcodeConfig, BarcodeLocationType, BarcodeSeqKitConfig

# Define barcode configurations
barcode_5prime = BarcodeConfig(
    sequence="TCGCGAGGC",
    location=BarcodeLocationType.FIVE_PRIME,
    name="5",
    description="5' barcode"
)

barcode_3prime = BarcodeConfig(
    sequence="GGCCGGCCGG",
    location=BarcodeLocationType.THREE_PRIME,
    name="3",
    description="3' barcode"
)

# Create configuration
config = BarcodeSeqKitConfig(
    barcodes=[barcode_5prime, barcode_3prime],
    output_prefix="example",
    output_dir="./output",
    merge_orientations=False,
    keep_unmatched=True,
    verbose=True,
    search_softclipped=True,
    max_mismatches=0
)

# To actually run the processing, uncomment these lines and provide a valid BAM file path
# bam_file = "path/to/your/input.bam"
# stats = extract_barcodes_from_bam(bam_file, config)

## Conclusion
This notebook implements the BAM processing functionality for BarcodeSeqKit. It provides efficient extraction and classification of barcoded reads from BAM files without using abstract classes, making the code more direct and easier to understand.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()